In [60]:
import itertools
from itertools import combinations, product
import collections
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
import statistics

In [692]:
def line_prettifier(line):
    double_cons = [["ξ", "κσ"], ["ζ", "δσ"], ["ψ", "πσ"]]
    line = line.lower()
    #καὶγὰρἐμοὶ τοῦτο μέλει, μὴ φιλότητʼ ἀπώσῃ.', 'τίςἂνθεῶν σοι τόνδʼ ἄριστον ἄνδρʼ ἰδεῖν
    accents = { #that's taken from cltk straight up!!! credit ἥκω Διὸς παῖς τήνδε Θηβαίων χθὀνα
            "έἐέἐέὲέἐἑἔἕἒἓἐέἐἑἒἓἔἕὲ": "ε",
            "υὐὺὺὐὺύΰύὺὐὑὔὕὒὓὺύὐύύὐὑὒὓὔὕὺῢΰῠῡ": "υ",
            "οόόόόὀὸόόὸὀὁὄὂὃὅὸόὅόὀὁὂὃὄὅὸ": "ο",
            "ὶιὶἰίὶὶὶὶίϊίίίίὶἰἱἴἵἲἳίίἰἱὶῒΐῐῑἲἳἴἵ": "ι",
            "άαααάἄἄἂἀὰἂἂἀὰάὰἀάάἄάάάὰἀἁἂἃἄἅᾳᾂᾃάἀἁἂἃἄἅὰᾰᾱᾳᾀᾁᾂᾃᾄᾅᾲᾴ": "α",
            "ἡἡῃὴῃὴήὴῆἥήὴήἠἡἢἣἥἤἧἦῆῄῂῇῃᾓᾒᾗᾖᾑᾐῆήῃήἠἡἢἣἤἥἦἧὴῆῃᾐᾑᾒᾓᾔᾕᾖᾗῂῇῄ": "η",
            "ῶώώώώώὼώὠὡὢὣὤὥὦὧῶῲῴῷῳᾧᾦᾢᾣᾤᾡᾠῶώὠὡὢὣὤὥὦὧὼῶῳᾠᾡᾢᾣᾤᾥᾦᾧῲῷῴ": "ω",
            "ῖῖἶἶἷῖἶἶἷῖῗῖ": "ῖ",
            "ἆἇᾷᾆᾇἆἇᾶᾆᾇᾷ": "ᾶ",
            "ῦὖῦὗῦῧὖὗῦ": "ῦ",
        }
    for item in double_cons:
        if item[0] in line:
            line = line.replace(item[0], item[1])
        else:
            pass
    for char in line:
        for key in accents:
            if char in key:
                accent = accents.get(key)
                if accent:
                    line = line.replace(char, accent)
            else:
                pass
    return(line)

In [2]:
def cola_initial_splitter(line):
    cola = []
    punct_cola = [',', '.', '·', "-", ";", "—", "|"]
    possible_splits = []
    for item in punct_cola:
        if item in line:
            line = line.replace(item, '?')
    if line not in cola:
        cola = line.split('?')
    return(cola)

In [4]:
def cola_combinations(cola):
    res1 = []
    res = []
    for item in cola:
        if item == '':
            cola.remove(item)
    for i, j in itertools.combinations(range(len(cola) + 1), 2):
        res1.append([cola[0:i], cola[i:j], cola[j:len(cola)]])
    for item in res1:
        for element in item:
            if element == []:
                item.remove(element)
            elif element == ['']:
                item.remove(element)
        if item not in res:
            res.append(item)
    return res

In [693]:
def line_splits(line):
    line = line_prettifier(line)
    full_punct = {'’', ',', '.', '·', '?', '—', '’', '’'}
    
    # Remove full punctuation from the line
    line_without_punct = ''.join([char for char in line if char not in full_punct])
    split = {"line": line, "cola": cola_combinations(cola_initial_splitter(line))}
    
    # Split the line into characters without spaces
    split["full_line"] = list(line_without_punct.replace(' ', ''))
    
    cols = []
    for element in split["cola"]:
        el = []
        for colon in element:
            full = ''.join(colon)
            if len(colon) == 1:
                full = colon[0]
            full = ''.join([char for char in full if char not in full_punct])
            el.append(list(full.replace(' ', '')))
        cols.append(el)
    
    split["full_cola"] = cols
    return split

In [6]:
def sonority(letter):
    flaps = ["ρ"]
    laterals = ["λ"]
    nasals = ["ν", "μ"]
    voiceless_fricatives = ["ς", "σ"]
    voiced_stops = ["δ", "γ", "β", "ζ"]
    voiceless_stops = ["π", "τ", "κ", "ξ", "ψ", "θ", "χ"]
    result = 0
    if letter in flaps:
        result += 1
    elif letter in laterals:
        result += 2
    elif letter in nasals:
        result += 3
    elif letter in voiceless_fricatives:
        result += 4
    elif letter in voiced_stops:
        result += 5
    elif letter in voiceless_stops:
        result += 6
    return result

In [26]:
def syllabify(cola):
    vowels = ["ε", "ι", "ο", "α", "η", "ω", "υ", "ῖ", "ᾶ", "ῦ"]
    single_consonants = ["ς", "ρ", "τ", "θ", "π", "σ", "δ", "φ", "γ", "ξ", "κ", "λ", "χ", "β", "ν", "μ"]
    diphtongs = ["αι", "αῖ", "ευ", "εῦ", "αυ", "αῦ", "οι", "οῖ", "ου", "οῦ", "ει", "εῖ", "υι", "υῖ", "ηῦ"]
    voiced_stops = ["β", "δ", "γ"]
    voiceless_stops = ["π", "τ", "κ", "φ", "χ", "θ"]
    all_stops = ["φ", "π", "τ", "κ", "β", "δ", "γ"]
    pairs = ["θθ", "κχ", "πφ", "τθ", "φφ", "χχ", "ππ", "ττ", "κκ", "ββ", "δδ", "γγ", "ρρ", "λλ", "νν", "μμ", "σσ", "σς"]
    liquids = ["ρ", "λ"]
    nasals = ["ν", "μ"]
    syllable = []
    result = {}
    a = 0
    while a == 0:
        item = cola[0]
        i = 0
        try:
            if item in vowels:
                if cola[i+1] in vowels: #V1V2
                    potential_diphtong = item+cola[i+1]
                    if potential_diphtong not in diphtongs: #V1\V2
                        syllable.append(item)
                        new_cola = cola[i+1:]
                        result["syllable"] = syllable
                        result["cola"] = new_cola
                        a = a+1
                    else: 
                        if cola[i+2] in vowels: #V1V2\V3 NB no need to check further, if there is three morae that is too much
                            syllable.append(item)
                            syllable.append(cola[i+1])
                            new_cola = cola[i+2:]
                            result["syllable"] = syllable
                            result["cola"] = new_cola
                            a = a+1
                        else: #V1V2C
                            if cola[i+3] in vowels: #V1V2\CV -- preference for open syllables
                                syllable.append(item)
                                syllable.append(cola[i+1])
                                new_cola = cola[i+2:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            else: #V1V2\CC
                                if cola[i+4] in vowels: #V1 V2 C1 C2 V
                                    potential_double = cola[i+2]+cola[i+3]
                                    if potential_double in pairs: #paired consonants -> V1V2C1\C2V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+2] == "γ" and cola[i+3] in nasals: #γ + nasal -- homosyllabic -> V1V2\C1C2V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        new_cola = cola[i+2:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+2] in voiceless_stops and cola[i+3] in liquids: #muta cum liquida -> V1V2\C1C2V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        new_cola = cola[i+2:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+2] in voiced_stops and cola[i+3] in liquids: #V1V2\C1C2V note this and upper are separated due to different degrees of representation -- ie this will be re-tested
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        new_cola = cola[i+2:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+2] in voiced_stops and cola[i+3] in nasals: #voiced stop+nasal -> V1V2C1/C2V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    else: #regular two consonant cluster -> V1V2C1/C2V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                else: #V1 V2 C1 C2 C3 V
                                    if cola[i+3] in all_stops and cola[i+4] in liquids: #basically an amped up muta cum liquida -> V1V2C1\C2C3V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+3] in voiceless_stops and cola[i+4] in nasals: #voiceless stop + nasal -> V1V2C1\C2C3V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+3] in voiced_stops and cola[i+4] in nasals:#voiced stop + nasal -> V1V2C1C2\C3V
                                        if cola[i+2] != cola[i+3]:
                                            if sonority(cola[i+2]) <= sonority(cola[i+3]):
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                new_cola = cola[i+4:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            else:
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                new_cola = cola[i+3:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                        else:
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            new_cola = cola[i+3:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                    else: #all other combinations -> V1V2C1C2\C3V
                                        if cola[i+2] != cola[i+3]:
                                            if sonority(cola[i+2]) <= sonority(cola[i+3]):
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                new_cola = cola[i+4:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            else:
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                new_cola = cola[i+3:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                        else:
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            new_cola = cola[i+3:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                else: #VC
                    if cola[i+2] in vowels: #VCV -> preference for open syllables V/CV 
                        syllable.append(item)
                        new_cola = cola[i+1:]
                        result["syllable"] = syllable
                        result["cola"] = new_cola
                        a = a+1
                    else: #VCC
                        if cola[i+3] in vowels: #VCCV
                            potential_double = cola[i+1]+cola[i+2]
                            if potential_double in pairs: #paired consonants -> VC1\C2V
                                syllable.append(item)
                                syllable.append(cola[i+1])
                                new_cola = cola[i+2:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            elif cola[i+1] == "γ" and cola[i+2] in nasals: #γ + nasal -- homosyllabic -> V\C1C2V
                                syllable.append(item)
                                new_cola = cola[i+1:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            elif cola[i+1] in voiceless_stops and cola[i+2] in liquids: #muta cum liquida -> V\C1C2V
                                syllable.append(item)
                                new_cola = cola[i+1:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            elif cola[i+1] in voiced_stops and cola[i+2] in liquids: #V\C1C2V note this and upper are separated due to different degrees of representation -- ie this will be re-tested
                                syllable.append(item)
                                new_cola = cola[i+1:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            elif cola[i+1] in voiced_stops and cola[i+2] in nasals: #voiced stop+nasal -> VC1/C2V
                                syllable.append(item)
                                syllable.append(cola[i+1])
                                new_cola = cola[i+2:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            else: #regular two consonant cluster -> VC1/C2V
                                syllable.append(item)
                                syllable.append(cola[i+1])
                                new_cola = cola[i+2:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                        else: #VCCCV
                            if cola[i+2] in all_stops and cola[i+3] in liquids: #basically an amped up muta cum liquida -> VC1\C2C3V
                                syllable.append(item)
                                syllable.append(cola[i+1])
                                new_cola = cola[i+2:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            elif cola[i+2] in voiceless_stops and cola[i+3] in nasals: #voiceless stop + nasal -> VC1\C2C3V
                                syllable.append(item)
                                syllable.append(cola[i+1])
                                new_cola = cola[i+2:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            elif cola[i+2] in voiced_stops and cola[i+3] in nasals: #voiced stop + nasal -> VC1C2\C3V
                                if cola[i+1] != cola[i+2]:
                                    if sonority(cola[i+1]) <= sonority(cola[i+2]):
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    else:
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        new_cola = cola[i+2:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                else:
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    new_cola = cola[i+2:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                            else: #all other combinations -> VC1C2\C3V
                                if cola[i+1] != cola[i+2]:
                                    if sonority(cola[i+1]) <= sonority(cola[i+2]):
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    else:
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        new_cola = cola[i+2:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                else:
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    new_cola = cola[i+2:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1 
            else: 
                if cola[i+1] in vowels: #CV
                    if cola[i+2] in vowels: #CV1V2
                        potential_diphtong = cola[i+1]+cola[i+2]
                        if potential_diphtong not in diphtongs: #CV1/V2
                            syllable.append(cola[0])
                            syllable.append(cola[1])
                            new_cola = cola[i+2:]
                            result["syllable"] = syllable
                            result["cola"] = new_cola
                            a = a+1
                        else:
                            if cola[i+3] in vowels: #CV1V2/V3
                                syllable.append(item)
                                syllable.append(cola[i+1])
                                syllable.append(cola[i+2])
                                new_cola = cola[i+3:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            else: #CV1V2C
                                if cola[i+4] in vowels: #CV1V2CV preference for open syllables: CV1V2/CV
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    syllable.append(cola[i+2])
                                    new_cola = cola[i+3:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                                else: #CV1V2CC
                                    if cola[i+5] in vowels: #CV1V2CCV
                                        potential_double = cola[i+3]+cola[i+4]
                                        if potential_double in pairs: #paired consonants -> CV1V2C1\C2V
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            syllable.append(cola[i+3])
                                            new_cola = cola[i+4:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                        elif cola[i+3] == "γ" and cola[i+4] in nasals: #γ + nasal -- homosyllabic -> CV1V2\C1C2V
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            new_cola = cola[i+3:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                        elif cola[i+3] in voiceless_stops and cola[i+4] in liquids: #muta cum liquida -> CV1V2\C1C2V
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            new_cola = cola[i+3:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                        elif cola[i+3] in voiced_stops and cola[i+4] in liquids: #CV1V2\C1C2V note this and upper are separated due to different degrees of representation -- ie this will be re-tested
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            new_cola = cola[i+3:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                        elif cola[i+3] in voiced_stops and cola[i+4] in nasals: #voiced stop+nasal -> CV1V2C1/C2V
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            syllable.append(cola[i+3])
                                            new_cola = cola[i+4:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                        else: #regular two consonant cluster -> CV1V2C1/C2V
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            syllable.append(cola[i+3])
                                            new_cola = cola[i+4:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                    else: #CV1V2C1C2C3V
                                        if cola[i+4] in all_stops and cola[i+5] in liquids: #basically an amped up muta cum liquida -> CV1V2C1\C2C3V
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            syllable.append(cola[i+3])
                                            new_cola = cola[i+4:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                        elif cola[i+4] in voiceless_stops and cola[i+5] in nasals: #voiceless stop + nasal -> CV1V2C1\C2C3V
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            syllable.append(cola[i+3])
                                            new_cola = cola[i+4:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                        elif cola[i+4] in voiced_stops and cola[i+5] in nasals: #voiced stop + nasal -> CV1V2C1C2\C3V
                                            if cola[i+3] != cola[i+4]:
                                                if sonority(cola[i+3]) <= sonority(cola[i+4]):
                                                    syllable.append(item)
                                                    syllable.append(cola[i+1])
                                                    syllable.append(cola[i+2])
                                                    syllable.append(cola[i+3])
                                                    syllable.append(cola[i+4])
                                                    new_cola = cola[i+5:]
                                                    result["syllable"] = syllable
                                                    result["cola"] = new_cola
                                                    a = a+1
                                                else:
                                                    syllable.append(item)
                                                    syllable.append(cola[i+1])
                                                    syllable.append(cola[i+2])
                                                    syllable.append(cola[i+3])
                                                    new_cola = cola[i+4:]
                                                    result["syllable"] = syllable
                                                    result["cola"] = new_cola
                                                    a = a+1
                                            else:
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                new_cola = cola[i+4:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1 
                                        else: #all other combinations -> CV1V2C1C2\C3V
                                            if cola[i+3] != cola[i+4]:
                                                if sonority(cola[i+3]) <= sonority(cola[i+4]):
                                                    syllable.append(item)
                                                    syllable.append(cola[i+1])
                                                    syllable.append(cola[i+2])
                                                    syllable.append(cola[i+3])
                                                    syllable.append(cola[i+4])
                                                    new_cola = cola[i+5:]
                                                    result["syllable"] = syllable
                                                    result["cola"] = new_cola
                                                    a = a+1
                                                else:
                                                    syllable.append(item)
                                                    syllable.append(cola[i+1])
                                                    syllable.append(cola[i+2])
                                                    syllable.append(cola[i+3])
                                                    new_cola = cola[i+4:]
                                                    result["syllable"] = syllable
                                                    result["cola"] = new_cola
                                                    a = a+1
                                            else:
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                new_cola = cola[i+4:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1 
                    else: #CVC..
                        if cola[i+3] in vowels: #CVCV
                            syllable.append(cola[0])
                            syllable.append(cola[1])
                            new_cola = cola[i+2:]
                            result["syllable"] = syllable
                            result["cola"] = new_cola
                            a = a+1
                        else: #CVCC
                            if cola[i+4] in vowels: #CVCCV
                                potential_double = cola[i+2]+cola[i+3]
                                if potential_double in pairs: #paired consonants -> CVC1\C2V
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    syllable.append(cola[i+2])
                                    new_cola = cola[i+3:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                                elif cola[i+2] == "γ" and cola[i+3] in nasals: #γ + nasal -- homosyllabic -> CV\C1C2V
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    new_cola = cola[i+2:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                                elif cola[i+2] in voiceless_stops and cola[i+3] in liquids: #muta cum liquida -> CV\C1C2V
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    new_cola = cola[i+2:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                                elif cola[i+2] in voiced_stops and cola[i+3] in liquids: #CV\C1C2V note this and upper are separated due to different degrees of representation -- ie this will be re-tested
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    new_cola = cola[i+2:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                                elif cola[i+2] in voiced_stops and cola[i+3] in nasals: #voiced stop+nasal -> CVC1/C2V
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    syllable.append(cola[i+2])
                                    new_cola = cola[i+3:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                                else: #regular two consonant cluster -> CVC1/C2V
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    syllable.append(cola[i+2])
                                    new_cola = cola[i+3:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                            else: #CVCCCV
                                if cola[i+3] in all_stops and cola[i+4] in liquids: #basically an amped up muta cum liquida -> CVC1\C2C3V
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    syllable.append(cola[i+2])
                                    new_cola = cola[i+3:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                                elif cola[i+3] in voiceless_stops and cola[i+4] in nasals: #voiceless stop + nasal -> CVC1\C2C3V
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    syllable.append(cola[i+2])
                                    new_cola = cola[i+3:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                                elif cola[i+3] in voiced_stops and cola[i+4] in nasals: #voiced stop + nasal -> CVC1C2\C3V
                                    if cola[i+2] != cola[i+3]:
                                        if sonority(cola[i+2]) <= sonority(cola[i+3]):
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            syllable.append(cola[i+3])
                                            new_cola = cola[i+4:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                        else:
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            new_cola = cola[i+3:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                    else:
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                else: #all other combinations -> CVC1C2\C3V
                                    if cola[i+2] != cola[i+3]:
                                        if sonority(cola[i+2]) <= sonority(cola[i+3]):
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            syllable.append(cola[i+3])
                                            new_cola = cola[i+4:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                        else:
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            new_cola = cola[i+3:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                    else:
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                else: #CC
                    if cola[i+2] in vowels: #CCV
                        if cola[i+3] in vowels: #CCV1V2
                            potential_diphtong = cola[i+2]+cola[i+3]
                            if potential_diphtong not in diphtongs: #CCV1/V2
                                syllable.append(item)
                                syllable.append(cola[i+1])
                                syllable.append(cola[i+2])
                                new_cola = cola[i+3:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            else:
                                if cola[i+4] in vowels: #CCV1V2/V3
                                    syllable.append(item)
                                    syllable.append(cola[i+1])
                                    syllable.append(cola[i+2])
                                    syllable.append(cola[i+3])
                                    new_cola = cola[i+4:]
                                    result["syllable"] = syllable
                                    result["cola"] = new_cola
                                    a = a+1
                                else: #CCV1V2C
                                    if cola[i+5] in vowels: #CCV1V2CV preference for open syllables: CCV1V2/CV
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        syllable.append(cola[i+3])
                                        new_cola = cola[i+4:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    else: #CCV1V2CC:
                                        if cola[i+6] in vowels: #CCV1V2C1C2V:
                                            potential_double = cola[i+4]+cola[i+5]
                                            if potential_double in pairs: #paired consonants -> CCV1V2C1\C2V
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                syllable.append(cola[i+4])
                                                new_cola = cola[i+5:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            elif cola[i+4] == "γ" and cola[i+5] in nasals: #γ + nasal -- homosyllabic -> CCV1V2\C1C2V
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                new_cola = cola[i+4:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            elif cola[i+4] in voiceless_stops and cola[i+5] in liquids: #muta cum liquida -> CCV1V2\C1C2V
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                new_cola = cola[i+4:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            elif cola[i+4] in voiced_stops and cola[i+5] in liquids: #CCV1V2\C1C2V note this and upper are separated due to different degrees of representation -- ie this will be re-tested
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                new_cola = cola[i+4:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            elif cola[i+4] in voiced_stops and cola[i+5] in nasals: #voiced stop+nasal -> CCV1V2C1/C2V
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                syllable.append(cola[i+4])
                                                new_cola = cola[i+5:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            else: #regular two consonant cluster -> CCV1V2C1/C2V
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                syllable.append(cola[i+4])
                                                new_cola = cola[i+5:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                        else: #CCV1V2C1C2C3V
                                            if cola[i+5] in all_stops and cola[i+6] in liquids: #basically an amped up muta cum liquida -> CCV1V2C1\C2C3V
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                syllable.append(cola[i+4])
                                                new_cola = cola[i+5:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            elif cola[i+5] in voiceless_stops and cola[i+6] in nasals: #voiceless stop + nasal -> CCV1V2C1\C2C3V
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                syllable.append(cola[i+4])
                                                new_cola = cola[i+5:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            elif cola[i+5] in voiced_stops and cola[i+6] in nasals: #voiced stop + nasal -> CCV1V2C1C2\C3V
                                                if cola[i+4] != cola[i+5]:
                                                    if sonority(cola[i+4]) <= sonority(cola[i+5]):
                                                        syllable.append(item)
                                                        syllable.append(cola[i+1])
                                                        syllable.append(cola[i+2])
                                                        syllable.append(cola[i+3])
                                                        syllable.append(cola[i+4])
                                                        syllable.append(cola[i+5])
                                                        new_cola = cola[i+6:]
                                                        result["syllable"] = syllable
                                                        result["cola"] = new_cola
                                                        a = a+1
                                                    else:
                                                        syllable.append(item)
                                                        syllable.append(cola[i+1])
                                                        syllable.append(cola[i+2])
                                                        syllable.append(cola[i+3])
                                                        syllable.append(cola[i+4])
                                                        new_cola = cola[i+5:]
                                                        result["syllable"] = syllable
                                                        result["cola"] = new_cola
                                                        a = a+1
                                                else:
                                                    syllable.append(item)
                                                    syllable.append(cola[i+1])
                                                    syllable.append(cola[i+2])
                                                    syllable.append(cola[i+3])
                                                    syllable.append(cola[i+4])
                                                    new_cola = cola[i+5:]
                                                    result["syllable"] = syllable
                                                    result["cola"] = new_cola
                                                    a = a+1
                                            else: #all other combinations -> CCV1V2C1C2\C3V
                                                if cola[i+4] != cola[i+5]:
                                                    if sonority(cola[i+4]) <= sonority(cola[i+5]):
                                                        syllable.append(item)
                                                        syllable.append(cola[i+1])
                                                        syllable.append(cola[i+2])
                                                        syllable.append(cola[i+3])
                                                        syllable.append(cola[i+4])
                                                        syllable.append(cola[i+5])
                                                        new_cola = cola[i+6:]
                                                        result["syllable"] = syllable
                                                        result["cola"] = new_cola
                                                        a = a+1
                                                    else:
                                                        syllable.append(item)
                                                        syllable.append(cola[i+1])
                                                        syllable.append(cola[i+2])
                                                        syllable.append(cola[i+3])
                                                        syllable.append(cola[i+4])
                                                        new_cola = cola[i+5:]
                                                        result["syllable"] = syllable
                                                        result["cola"] = new_cola
                                                        a = a+1
                                                else:
                                                    syllable.append(item)
                                                    syllable.append(cola[i+1])
                                                    syllable.append(cola[i+2])
                                                    syllable.append(cola[i+3])
                                                    syllable.append(cola[i+4])
                                                    new_cola = cola[i+5:]
                                                    result["syllable"] = syllable
                                                    result["cola"] = new_cola
                                                    a = a+1
                        else: #CCVC..
                            if cola[i+4] in vowels: #CCVCV
                                syllable.append(item)
                                syllable.append(cola[i+1])
                                syllable.append(cola[i+2])
                                new_cola = cola[i+3:]
                                result["syllable"] = syllable
                                result["cola"] = new_cola
                                a = a+1
                            else: #CСVCC
                                if cola[i+5] in vowels: #CСVCCV
                                    potential_double = cola[i+3]+cola[i+4]
                                    if potential_double in pairs: #paired consonants -> CСVC1\C2V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        syllable.append(cola[i+3])
                                        new_cola = cola[i+4:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+3] == "γ" and cola[i+4] in nasals: #γ + nasal -- homosyllabic -> CСV\C1C2V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+3] in voiceless_stops and cola[i+4] in liquids: #muta cum liquida -> СCV\C1C2V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+3] in voiced_stops and cola[i+4] in liquids: #CСV\C1C2V note this and upper are separated due to different degrees of representation -- ie this will be re-tested
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        new_cola = cola[i+3:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+3] in voiced_stops and cola[i+4] in nasals: #voiced stop+nasal -> СCVC1/C2V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        syllable.append(cola[i+3])
                                        new_cola = cola[i+4:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    else: #regular two consonant cluster -> СCVC1/C2V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        syllable.append(cola[i+3])
                                        new_cola = cola[i+4:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                else: #CCVCCCV
                                    if cola[i+4] in all_stops and cola[i+5] in liquids: #basically an amped up muta cum liquida -> CCVC1\C2C3V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        syllable.append(cola[i+3])
                                        new_cola = cola[i+4:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+4] in voiceless_stops and cola[i+5] in nasals: #voiceless stop + nasal -> CCVC1\C2C3V
                                        syllable.append(item)
                                        syllable.append(cola[i+1])
                                        syllable.append(cola[i+2])
                                        syllable.append(cola[i+3])
                                        new_cola = cola[i+4:]
                                        result["syllable"] = syllable
                                        result["cola"] = new_cola
                                        a = a+1
                                    elif cola[i+4] in voiced_stops and cola[i+5] in nasals: #voiced stop + nasal -> CCVC1C2\C3V
                                        if cola[i+3] != cola[i+4]:
                                            if sonority(cola[i+3]) <= sonority(cola[i+4]):
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                syllable.append(cola[i+4])
                                                new_cola = cola[i+5:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            else:
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                new_cola = cola[i+4:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                        else:
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            syllable.append(cola[i+3])
                                            new_cola = cola[i+4:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
                                    else: #all other combinations -> CCV1V2C1C2\C3V
                                        if cola[i+3] != cola[i+4]:
                                            if sonority(cola[i+3]) <= sonority(cola[i+4]):
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                syllable.append(cola[i+4])
                                                new_cola = cola[i+5:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                            else:
                                                syllable.append(item)
                                                syllable.append(cola[i+1])
                                                syllable.append(cola[i+2])
                                                syllable.append(cola[i+3])
                                                new_cola = cola[i+4:]
                                                result["syllable"] = syllable
                                                result["cola"] = new_cola
                                                a = a+1
                                        else:
                                            syllable.append(item)
                                            syllable.append(cola[i+1])
                                            syllable.append(cola[i+2])
                                            syllable.append(cola[i+3])
                                            new_cola = cola[i+4:]
                                            result["syllable"] = syllable
                                            result["cola"] = new_cola
                                            a = a+1
        except IndexError as e:
            result["syllable"] = cola
            result["cola"] = []
            a = a+1
                
                
    return(result)

In [457]:
def cola_syllabifier(cola):
    syllables = []
    while cola:
        res = syllabify(cola)
        cola = res["cola"]
        syllables.append(res["syllable"])
    return syllables

In [497]:
def syllable_weight(syllables):
    vowels = ["ε", "ι", "ο", "α", "η", "ω", "υ", "ῖ", "ᾶ", "ῦ"]
    weird = ["b"]
    long_vowels = ["η", "ω", "ῖ", "ᾶ", "ῦ"]
    weights = []
    for item in syllables:
        i = syllables.index(item)
        v = 0
        if i < len(syllables) - 1:
            for element in item:
                if element in vowels:
                    v += 1
            if v > 1:
                weights.append("-")
            else:
                for element in item:
                    if element in long_vowels:
                        weights.append("-")
                    else:
                        if element in vowels:
                            n = item.index(element)
                            if n == len(item) - 1:
                                if element in weird:
                                    weights.append("x")
                                else:
                                    weights.append("u")
                            else:
                                weights.append("-")
        else:
            for element in item:
                if element in vowels:
                    v += 1
            if v > 1:
                weights.append("-")
            else:
                for element in item:
                    if element in long_vowels:
                        weights.append("-")
                    else:
                        if element in vowels:
                            n = item.index(element)
                            if n == len(item) - 1:
                                weights.append("x")
                            else:
                                weights.append("-")
    return(weights)

In [445]:
def resolutions(l):
    result = []
    for i, item in enumerate(l):
        if item == 'x':
            new1 = l[:]
            new1[i] = 'u'
            result.append(new1)
            new2 = l[:]
            new2[i] = '-'
            result.append(new2)
    return result

In [443]:
def fin_anceps(abra):
    fin = []
    for item in abra:
        res = resolutions(item)
        for item in res:
            if item not in abra:
                abra.append(item)
    for item in abra:
        if 'x' in item:
            abra.remove(item)
        else:
            fin.append(item)
    return(fin)

In [11]:
def sublist_gen(ls):
    n = len(ls)
    if n == 1:
        yield [ls]
    else:
        for left in sublist_gen(ls[0:n//2]):
            for right in sublist_gen(ls[n//2:n]):
                yield left + right
                yield left[0:-1] + [left[-1] + right[0]] + right[1:]

In [436]:
def final_picker(list):
    result1 = []
    result = []
    for sublist in list:
        l = len(sublist)
        a = 0
        for item in sublist:
            if sublist.index(item) != l-1:
                if len(item) == 1:
                    a += 1
                else:
                    pass
        if a == 0:
            result1.append(sublist)
    for item in result1:
        new_it = []
        for element in item:
            try:
                new_it.append("".join(element))
            except TypeError:
                pass
        result.append(new_it)
    return(result)


In [621]:
def all_substitutions(string):
    if '-' not in string and 'u' not in string:
        return [string]

    substitutions = []

    for i, char in enumerate(string):
        if char == '-':
            substitutions.extend(all_substitutions(string[:i] + 'x' + string[i+1:]))
        elif char == 'u':
            substitutions.extend(all_substitutions(string[:i] + 'x' + string[i+1:]))
    
    substitutions.append(string)

    return list(set(substitutions))

In [683]:
def metre_initial(l):
    meter_types = {
        'iambic': ['--', 'u-u-', 'uuuu-', 'u-uuu', 'uuuuuu', '-u-', 'u--', '---', 'uuuu', '--u-'],
        'trochaic': ['--', '-u-u', 'uuu-u', '-uuuu', 'uuuuuu', '-u-', 'u--', '---', 'uuuu'],
        'cretic_paeonic': ['-u-', '-uuu', 'uuu-', 'u--', '--u', '---'],
        'dactylic': ['-uu', 'uuuu', '-u', '--'],
        'anapaestic': ['--', 'uu-', 'uuuu', 'uu--u-', 'uu--uuuu', 'uu----', 'uu--uu-', "--uu-", "--uuuu", "----", "---u-", 'uu-uu-', 'uu-uuuu', 'uu---', 'uu--u-', 'uuuuuuuu', 'uuuuuu-', 'uuuu--', 'uuuu-u-', '-u-uu-', '-u-uuuu', '-u---', '-u--u-'],
        'dochmiac': ['u--u-', 'u--u-', 'uuu-u-', 'u-uuu-', 'u--u-', 'u--uuu', 'uuuuuu-', 'uuu-uuu', 'u-uuuuu', 'uuuuuuuu', 'uu--u-', '-uu-u-', '--uuu-', '---u-', '---uuu', 'uuuu-u-', 'uu-uuu-', 'uu--uuu', '-uuuuu-', '-uu-uuu', '--uuuuu', 'uuuuuuu-', 'uuuu-uuu', 'uu-uuuuu', '-uuuuuuu', 'uuuuuuuuu', 'u----', 'uuu---', 'u-uu--', 'u--uu-', 'u---uu', 'uuuuu--', 'uuu-uu-', 'uuu--uu', 'u-uuuu-', 'u-uu-uu', 'u--uuuu', 'uuuuuuu-', 'uuuuu-uu', 'uuu-uuuu', 'u-uuuuuu', 'uuuuuuuuu', 'uu----', '-uu---', '--uu--', '---uu-', '----uu', 'uuuu---', 'uu-uu--', 'uu--uu-', 'uu---uu', '-uuuu--', '-uu-uu-', '-uu--uu', '--uuuu-', '--uu-uu', '---uuuu', 'uuuuuu--', 'uuuu-uu-', 'uuuu--uu', 'uu-uuuu-', 'uu-uu-uu', 'uu--uuuu', '-uuuuuu-', '-uuuu-uu', '-uu-uuuu', '--uuuuuu', 'uuuuuuuu-', 'uuuuuu-uu', 'uuuu-uuuu', 'uu-uuuuuu', '-uuuuuuuu', 'uuuuuuuuuu'],
        'hypodochmius': ['-u-u-', '-uuuu-', '-u-uuu', '-u---'],
        'iambo_trochaic': ['-u-u-', 'u-u-u', '--u-u', 'u-u--', '--u--'],
        'ionic': ['uu--', 'uu-u', '-u--', '-u-', '--', 'uuuu-', 'uu-uu', 'uuuuuu', 'uuuuu', 'uuu--', '-uuu-', '-u-uu', 'uuuuu-', 'uuu-uu', '-uuuuu', 'uuuuuuu', 'u-u--'],
        'choriamb': ['-uu-'],
        'aeolo_choriambic': ['---uu-u-', '-u-uu-u-', 'u--uu-u-', 'uuu-uu-u-', '-uu-uu-u-', 'uu--uu-u-', '---uu---', '-u-uu---', 'u--uu---', 'uuu-uu---', '-uu-uu---', 'uu--uu---', '-uu-u-u-', '-u-u-uu-', '---u-uu-', '-----uu-', '-u---uu-', '-uu-u--', 'u-uu-u--', '--uu-u--', 'u-u-uu--', '--u-uu--', 'u---uu--', '----uu--', 'u-uu-u-', '--uu-u-', 'u-uu---', '--uu---', 'u-uu--', '--uu--', 'u-u-uu-', 'u---uu-', '--u-uu-', '----uu-', '-uu-u-', '-uu---', '-u-uu-', '---uu-', 'u--uu-', '---uu-', '-uu-u-u--', '-u-uu-u--', '---uu-u--', 'u-u--uu--', '--u--uu--', '-uu--uu--', 'u-uu-u-u-', '--uu-u-u-', 'u-uu-u---', '--uu-u---', 'u-u-uu-u-', 'u---uu-u-', '--u-uu-u-', '----uu-u-', 'uu-uu-u-u-', 'uu-uu-u---', 'u--uu-u-u-', 'u--uu-u---', '-u-uu-u-u-', '-u-uu-u---', '---uu-u-u-', '---uu-u---', '-u-u-uu-u-', '-u-u-uu---', '---u-uu-u-', '---u-uu---', '-u-uu--uu--', '---uu--uu-', 'u-uu-u-u--', '--uu-u-u--', 'u-u-uu-u--', 'u---uu-u--', '--u-uu-u--', '----uu-u--', '-u-uu-u-u--', '---uu-u-u--', '-u-uu-u----', 'u--u-uu-u--', '-u-u-uu-u--', '-u---uu-u--', '-u-uu--uu--', '---uu--uu--', 'u-u-u-uu-u-', 'u-u---uu-u-', '--u-u-uu-u-', '--u---uu-u-', 'u-u-uu-u-u-', 'u---uu-u-u-', '--u-uu-u-u-', '----uu-u-u-', 'u-u-uu-u---', 'u---uu-u---', '--u-uu-u---', '----uu-u---', '-uu-u-u-u-u-', '-uu-u-u---u-', '-u-uu-u-u-u-', '---uu-u-u-u-', 'u-u--u-uu-u-', 'u-u----uu-u-', '--u--u-uu-u-', '--u----uu-u-', '-uu-u--uu-u-', '-u-uu--uu-u-', '-u-uu--uu---', '---uu--uu-u-', '---uu--uu---', '-uu-u-u--uu-', 'u--u--uu-u--', 'u-uu--uu-u--', '--uu--uu-u--', 'u-u-uu--uu--', '--u-uu--uu--'],
        'dactylo_epitrite': ['-u--uu-uu', 'u-u--uu-uu', '--u--uu-uu', '-u-u-uu-uu', '-u---uu-uu', '-u--uu-uuu', 'u-u--uu-uu-', 'u-u-u-uu-uu', 'u-u---uu-uu', 'u-u--uu-uuu', 'u-u--uu-uu-', '--u-u-uu-uu', '--u---uu-uu', '--u--uu-uuu', '--u--uu-uu-', '-u-u-uu-uuu', '-u---uu-uuu', '-u-u-uu-uu-', '-u---uu-uu-', 'u-u-u-uu-uu-u', 'u-u---uu-uu-u', 'u-u-u-uu-uu--', '--u-u-uu-uu-u', 'u-u---uu-uu--', '--u-u-uu-uu--', '--u---uu-uu-u', '--u---uu-uu--', '-u--uuu-uu--', '-uuu-uu-uu---uuuuuu', '-u---uu-uu---u-'], 
        'lekythion': ['-u---u-', '-u-u-u-'],
        'ithyphallic': ['-u-u--'],
        'final': ['-', 'uu', 'u']
    }

    result = []

    for meter_type, patterns in meter_types.items():
        if l in patterns:
            result.append(meter_type)

    if not result:
        result.append("NO")

    return result

In [428]:
def metre_entire_line(dict_line):
    return [[metre_initial(poda) for poda in item] for item in dict_line]

In [426]:
def metre_cleaner(ls):
    res = []
    for item in ls:
        i = ls.index(item)
        if ['NO'] not in item:
            if ['final'] in item:
                if item not in res:
                    res.append(item)
            else:
                if item not in res:
                    res.append(item)
    return(res)

In [411]:
def internal_picker(var):
    result = []
    for item in var:
        r = list(product(*item))
        result.append(r)
    res = list(itertools.chain(*result))
    return res


In [406]:
def line_metre_weeder(var):
    result = []
    combinations = {
        "dactylic": ["dactylo-epitrite", "iambic", "trochaic", "iambo-trochaic"],
        "anapaestic": ["dochmiac", "dactylic", "choriambic"],
        "iambic": ["trochaic", "iambo-trochaic", "lekythion", "ithyphallic", "choriambic", "cretic_paeonic"],
        "trochaic": ["iambic", "iambo-trochaic", "cretic_paeonic", "lekythion"],
        "cretic_paeonic": ["iambic", "trochaic", "iambo-trochaic", "dochmiac"],
        "dochmiac": ["iambic", "cretic_paeonic", "lekythion", "hypodochmius"],
        "ionic": ["aeolic", "anapaestic"],
        "aeolo_choriambic": ["choriambic", "iambic", "dactylic"],
        "dactylo_epitrite": ["iambic", "trochaic", "anapaestic", "dactylic"]
    }
    
    for item in var:
        l = list(item)
        if len(set(l)) == 1:  # All elements are the same
            result.append(l)
        else:  # Deleting 'final' and filtering for combinations
            neue = [x for x in l if x != 'final']
            if len(set(neue)) == 1:  # All elements are the same sans 'final'
                result.append(l)
            else:  # Checking for combinations
                check1, check2 = neue[:2]
                if check1 in combinations and check2 in combinations[check1]:
                    result.append(l)

    return result


In [516]:
def strip_finals(var):
    res = []
    for item in var:
        if 'final' not in item:
            res.append(item)
        else:
            new = []
            for element in item:
                if element != 'final':
                    new.append(element)
                else:
                    pass
            res.append(new)
    return(res)

In [672]:
def line_single_likelihood(var):
    closeness1 = ['iambic', 'trochaic', 'iambo-trochaic', 'spondaic']
    closeness2 = ['dactylic', 'anapaestic', 'dactylo-epitrite', 'spondaic']
    closeness3 = ['dochmiac', 'hypodochmius', 'spondaic']
    result = {}
    result['same'] = [] 
    result['same+fin'] = [] 
    result['same+2fin'] = [] 
    result['same+rand+close'] = [] 
    result['same+rand+fin+close'] = [] 
    result['same+rand+2fin+close'] = [] 
    result['same+rand'] = [] 
    result['same+rand+fin'] = [] 
    result['same+rand+2fin'] = []
    result['most+close'] = [] 
    result['most'] = [] 
    result['half+close'] = [] 
    result['half+1+close'] = []
    result['half+2+close'] = [] 
    result['half'] = [] 
    result['half+1'] = []
    result['half+2'] = [] 
    result['else+close'] = []
    result['else'] = [] 
    exclude_keys = ['final']
    for item in var:
        counts = collections.Counter(item)
        if len(counts.keys()) == 1: #everything is the same
            result['same'].append({'same': item})
        elif len(counts.keys()) == 2: #2 metra 
            if 'final' in counts.keys(): #metre + finals
                if counts['final'] == 1:
                    result['same+fin'].append({'same+fin': item})
                elif counts['final'] == 2:
                    result['same+2fin'].append({'same+2fin': item})
            else: #2 separate metra no finals
                if 1 in counts.values(): #all the same but one random metron
                    i = []
                    for key in counts.keys():
                        if key in closeness1:
                            i.append(1)
                        elif key in closeness2:
                            i.append(2)
                        elif key in closeness3:
                            i.append(3)
                        else:
                            pass
                    if len(i) != 0:
                        ks = collections.Counter(i)
                        if len(ks.keys()) == 1:
                            result['same+rand+close'].append({'same+rand+close': item})
                        else:
                            result['same+rand'].append({'same+rand': item})
                    else:
                        result['same+rand'].append({'same+rand': item})                    
                elif list(counts.values())[0] == list(counts.values())[1]: #half and half
                    i = []
                    for key in counts.keys():
                        if key in closeness1:
                            i.append(1)
                        elif key in closeness2:
                            i.append(2)
                        elif key in closeness3:
                            i.append(3)
                        else:
                            pass
                    if len(i) != 0:
                        ks = collections.Counter(i)
                        if len(ks.keys()) == 1:
                            result['half+close'].append({'half+close': item})
                        else:
                            result['half'].append({'half': item})
                    else:
                        result['half'].append({'half': item})        
                else:
                    i = []
                    for key in counts.keys():
                        if key in closeness1:
                            i.append(1)
                        elif key in closeness2:
                            i.append(2)
                        elif key in closeness3:
                            i.append(3)
                        else:
                            pass
                    if len(i) != 0:
                        ks = collections.Counter(i)
                        if len(ks.keys()) == 1:
                            result['most+close'].append({'most+close': item})
                        else:
                            result['most'].append({'most': item})
                    else:
                        result['most'].append({'most': item})     #nothing else could be admissible
        elif len(counts.keys()) == 3: #3 metra -- to check for sames + rand + fin 
            if 'final' in counts.keys(): 
                new_d = {k: counts[k] for k in set(list(counts.keys())) - set(exclude_keys)}
                if counts['final'] == 1: #metre + finals + one rand
                    a = 0
                    for ct in counts.values():
                        if ct == 1:
                            a+=1
                    if a > 1:
                        i = []
                        for key in new_d.keys():
                            if key in closeness1:
                                i.append(1)
                            elif key in closeness2:
                                i.append(2)
                            elif key in closeness3:
                                i.append(3)
                            else:
                                pass
                        if len(i) != 0:
                            ks = collections.Counter(i)
                            if len(ks.keys()) == 1:
                                result['same+rand+fin+close'].append({'same+rand+fin+close': item})
                            else:
                                result['same+rand+fin'].append({'same+rand+fin': item})
                        else:
                            result['same+rand+fin'].append({'same+rand+fin': item})
                    else: 
                        if list(new_d.values())[0] == list(new_d.values())[1]: #half and half +fin
                            i = []
                            for key in new_d.keys():
                                if key in closeness1:
                                    i.append(1)
                                elif key in closeness2:
                                    i.append(2)
                                elif key in closeness3:
                                    i.append(3)
                                else:
                                    pass
                            if len(i) != 0:
                                ks = collections.Counter(i)
                                if len(ks.keys()) == 1:
                                    result['half+1+close'].append({'half+1+close': item})
                                else:
                                    result['half+1'].append({'half+1': item})
                            else:
                                result['half+1'].append({'half+1': item})
                        else:
                            i = []
                            for key in new_d.keys():
                                if key in closeness1:
                                    i.append(1)
                                elif key in closeness2:
                                    i.append(2)
                                elif key in closeness3:
                                    i.append(3)
                                else:
                                    pass
                            if len(i) != 0:
                                ks = collections.Counter(i)
                                if len(ks.keys()) == 1:
                                    result['most+close'].append({'most+close': item})
                                else:
                                    result['most'].append({'most': item})
                            else:
                                result['most'].append({'most': item})
                elif counts['final'] == 2:
                    if 1 in counts.values():
                        i = []
                        for key in new_d.keys():
                            if key in closeness1:
                                i.append(1)
                            elif key in closeness2:
                                i.append(2)
                            elif key in closeness3:
                                i.append(3)
                            else:
                                pass
                        if len(i) != 0:
                            ks = collections.Counter(i)
                            if len(ks.keys()) == 1:
                                result['same+rand+2fin+close'].append({'same+rand+2fin+close': item})
                            else:
                                result['same+rand+2fin'].append({'same+rand+2fin': item})
                        else:
                            result['same+rand+2fin'].append({'same+rand+2fin': item})
                    else:
                        if list(new_d.values())[0] == list(new_d.values())[1]: #half and half +2fin
                            i = []
                            for key in new_d.keys():
                                if key in closeness1:
                                    i.append(1)
                                elif key in closeness2:
                                    i.append(2)
                                elif key in closeness3:
                                    i.append(3)
                                else:
                                    pass
                            if len(i) != 0:
                                ks = collections.Counter(i)
                                if len(ks.keys()) == 1:
                                    result['half+2+close'].append({'half+2+close': item})
                                else:
                                    result['half+2'].append({'half+2': item})
                            else:
                                result['half+2'].append({'half+2': item})
                        else:
                            i = []
                            for key in new_d.keys():
                                if key in closeness1:
                                    i.append(1)
                                elif key in closeness2:
                                    i.append(2)
                                elif key in closeness3:
                                    i.append(3)
                                else:
                                    pass
                            if len(i) != 0:
                                ks = collections.Counter(i)
                                if len(ks.keys()) == 1:
                                    result['most+close'].append({'most+close': item})
                                else:
                                    result['most'].append({'most': item})
                            else:
                                result['most'].append({'most': item})
            else: #4 or more metra 
                new_d = {k: counts[k] for k in set(list(counts.keys())) - set(exclude_keys)}
                higher = [] #here trying to understand if there is a possibility for a dominating metre with random additions
                indices = [i for i in range(len(counts.values()))]
                for element in counts.values():
                    diff = []
                    for index in indices:
                        if index != list(counts.values()).index(element):
                            a = element - list(counts.values())[index]
                            diff.append(a)
                    res = []
                    for different in diff:
                        if different > 3:
                            res.append('yes')
                    if len(res) != 0:
                        if res.count(res[0]) == len(res): 
                            higher.append('+')
                if len(higher) == 1:
                    i = []
                    for key in new_d.keys():
                        if key in closeness1:
                            i.append(1)
                        elif key in closeness2:
                            i.append(2)
                        elif key in closeness3:
                            i.append(3)
                        else:
                            pass
                    if len(i) != 0:
                        ks = collections.Counter(i)
                        if len(ks.keys()) == 1:
                            result['most+close'].append({'most+close': item})
                        else:
                            result['most'].append({'most': item})
                    else:
                        result['most'].append({'most': item})
                else:
                    i = []
                    for key in new_d.keys():
                        if key in closeness1:
                            i.append(1)
                        elif key in closeness2:
                            i.append(2)
                        elif key in closeness3:
                            i.append(3)
                        else:
                            pass
                    if len(i) != 0:
                        ks = collections.Counter(i)
                        if len(ks.keys()) == 1:
                            result['else+close'].append({'else+close': item})
                        else:
                            result['else'].append({'else': item})
                    else:
                        result['else'].append({'else': item})
    final = []
    a = 0
    while a <= 1:
        if len(result['same']) >= 1:
            final.append(result['same'])
            a += 1
        elif len(result['same+fin']) >= 1:
            final.append(result['same+fin'])
            a += 1
        elif len(result['same+2fin']) >= 1:
            final.append(result['same+2fin'])
            a += 1
        elif len(result['same+rand+close']) >= 1:
            final.append(result['same+rand+close'])
            a += 1
        elif len(result['most+close']) >= 1:
            final.append(result['most+close'])
            a += 1
        elif len(result['half+close']) >= 1:
            final.append(result['half+close'])
            a += 1
        elif len(result['same+rand+fin+close']) >= 1:
            final.append(result['same+rand+fin+close'])
            a += 1
        elif len(result['same+rand+2fin+close']) >= 1:
            final.append(result['same+rand+2fin+close'])
            a += 1
        elif len(result['half+1+close']) >= 1:
            final.append(result['half+1+close'])
            a += 1
        elif len(result['half+2+close']) >= 1:
            final.append(result['half+2+close'])
            a += 1
        elif len(result['else+close']) >= 1:
            final.append(result['else+close'])
            a += 1
        elif len(result['same+rand']) >= 1:
            final.append(result['same+rand'])
            a += 1
        elif len(result['most']) >= 1:
            final.append(result['most'])
            a += 1
        elif len(result['half']) >= 1:
            final.append(result['half'])
            a += 1
        elif len(result['same+rand+fin']) >= 1:
            final.append(result['same+rand+fin'])
            a += 1
        elif len(result['same+rand+2fin']) >= 1:
            final.append(result['same+rand+2fin'])
            a += 1
        elif len(result['half+1']) >= 1:
            final.append(result['half+1'])
            a += 1
        elif len(result['half+2']) >= 1:
            final.append(result['half+2'])
            a += 1
        elif len(result['else']) >= 1:
            final.append(result['else'])
            a += 1
        
    fin = []
    for element in final:
        doubles = []
        for d in element:
            for value in d.values():
                doubles.append(' '.join(value))
        doub = collections.Counter(doubles)
        for key in doub.keys():
            fin.append(key.split(' '))
    return(fin)

In [22]:
def similarity_transformers(var):
    result = {}
    sentences = []
    for item in var:
        sentences.append(item)
    sentence_embeddings = model.encode(sentences)
    a = float(util.pytorch_cos_sim(sentence_embeddings[0]+sentence_embeddings[2], sentence_embeddings[1]))
    b = float(util.pytorch_cos_sim(sentence_embeddings[0]+sentence_embeddings[1], sentence_embeddings[2]))
    c = float(util.pytorch_cos_sim(sentence_embeddings[1]+sentence_embeddings[2], sentence_embeddings[0]))
    num = []
    num.append(a)
    num.append(b)
    num.append(c)
    res = statistics.fmean(num) 
    result[res] = var
    return(result)

In [611]:
def one_variable(var): #two lines return a single variant of analysis and one returns several
    result = []
    checker = []
    new_checker = []
    for item in var:
        if len(item) == 1: #creating the pool of unambiguous analyses to see if there is repeats
            new_checker.append(' '.join(item[0])) #two lists - one for checking perfect repeats, one for quantitative (e.g. [trochaic, trochaic], [trochaic, trochaic, trochaic], [[trochaic, trochaic, iambic], [iambic, iambic]])
            checker.append(item[0])
        else:
            checkee = item
            checker.append('a')
    ch = collections.Counter(new_checker) #checking if the two analyses are the same
    if len(ch.keys()) == 1: #if the two are the same
        new_checkee = [] #checking perfect repeats
        for item in checkee:
            new_checkee.append(' '.join(item))
        for chis in new_checkee:
            if chis in new_checker:
                for item in var:
                    i = var.index(item)
                    if len(item) == 1:
                        result.append(item[0])
                    else:
                        result.append(chis.split(' '))
    if len(result) != len(var):#continuing if no perfect repeats -> via cosine similarity w/ embeddings
        cos_check = []
        ind = checker.index('a')
        for item in checkee:
            check_list = []
            l = ' '.join(item)
            for element in checker:
                check_list.append(' '.join(element))
            try:
                check_list[ind] = l
            except IndexError:
                check_list.append(l)
            cos_check.append(check_list)
        to_pick = []
        for item in cos_check:
            to_pick.append(similarity_transformers(item))
        maxs = []
        for element in to_pick:
            for key in element.keys():
                maxs.append(key)
        mx_sim = max(maxs)
        for element in to_pick:
            for key in element.keys():
                if key == mx_sim:
                    res = element[key]
                    for item in res:
                        result.append(item.split(' '))
    return(result)


In [389]:
def two_variable(var):
    for_counter = [[' '.join(element) for element in item] for item in var]
    a = list(set.intersection(*[set(lst) for lst in for_counter]))
    if len(a) == 1:
        return [a[0].split(' ')] * 3
    i_result = []
    checkees = [[{i: ' '.join(element[i]) for i in range(len(element))} for element in item] for item in var]
    new_checker = ['a' in item for item in i_result]
    if any(new_checker):
        for_counter = [[element for sublist in item for element in sublist] for item in var]
        result = [[element.split(' ') for element in a] for a in for_counter]
    else:
        l_full = [[' '.join(element) for element in item] for item in internal_picker([var])]
        to_pick = [similarity_transformers(item) for item in l_full]
        maxs = [key for element in to_pick for key in element.keys()]
        mx_sim = max(maxs)
        result = [[item.split(' ') for item in res] for element in to_pick for key, res in element.items() if key == mx_sim]
    return result[:3]

In [475]:
def three_variable(var):
    for_counter = [[' '.join(element) for element in item] for item in var]
    intersection = set.intersection(*[set(lst) for lst in for_counter])

    if len(intersection) == 1:  # intersection through three
        result = [list(intersection)[0].split(' ') for _ in var]
    else:
        picks = internal_picker([var])
        l_full = [[' '.join(element) for element in sublist] for sublist in picks]
        to_pick = [similarity_transformers(item) for item in l_full]
        max_similarity = max(max(item.keys()) for item in to_pick)
        
        result = [item for element in to_pick for key, value in element.items() if key == max_similarity for item in value]

    return result[:3]

In [703]:
import itertools

def single_line_processing(line):
    result = {}
    test = line_splits(line)
    line_syllables = cola_syllabifier(test["full_line"])
    colae = [[cola_syllabifier(element) for element in item] for item in test["full_cola"]]
    line_weight = syllable_weight(line_syllables)
    cola_weights = [list(itertools.chain(*[syllable_weight(cola) for cola in element])) for element in colae]

    possibilities = [line_weight] + [item for sublist in [fin_anceps(resolutions(item)) or [item] for item in cola_weights] for item in sublist]
    cola_splits = [item for sublist in [final_picker(sublist_gen(item)) for item in possibilities] if isinstance(sublist, list) for item in sublist]
    metres_1 = metre_cleaner(metre_entire_line(cola_splits))
    rs = line_metre_weeder(internal_picker(metres_1))
    result['line'] = line
    a = strip_finals([['asynartetic']] if not rs else line_single_likelihood(rs))
    result['metre'] = [x for x in a if x != []]
    return result


In [704]:
def three_line_processing(var):
    result = []
    vars = []
    i = 0
    for item in var:
        vars.append(item['metre'])
        if len(item['metre']) > 1:
            i+=1
        else:
            pass
    if i == 0:
        for item in var:
            line = {}
            line['line'] = item['line']
            line['metre'] = item['metre'][0]
            result.append(line)
    elif i == 1:
        res = one_variable(vars)
        for item in var:
            l = var.index(item)
            line = {}
            line['line'] = item['line']
            line['metre'] = res[l]
            result.append(line)
    elif i == 2:
        xss = two_variable(vars)
        res = [x for xs in xss for x in xs]
        for item in var:
            l = var.index(item)
            line = {}
            line['line'] = item['line']
            line['metre'] = res[l]
            result.append(line)
    elif i == 3:
        res = three_variable(vars)
        for item in var:
            l = var.index(item)
            line = {}
            line['line'] = item['line']
            line['metre'] = res[l]
            result.append(line)
    rests = []
    for item in result:
        n = {}
        n['line'] = item['line']
        if type(item['metre']) == list:
            n['metre'] = item['metre']
        else:
            n['metre'] = item['metre'].split(' ')
        rests.append(n)
    return(rests)